In [1]:
import pandas as pd
import json
import numpy as np
import os
from google.cloud import storage
from io import BytesIO
from pypdf import PdfReader  # You may need to install pypdf if you haven't already
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
#from langchain.llms import OpenAI
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from langchain_google_vertexai import ChatVertexAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import logging

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'xenon-heading-430209-e4-7582f64a7330.json' # Rellenar con el .json correspondiente
logging.basicConfig(filename='error_log.txt', level=logging.ERROR, format='%(asctime)s %(message)s')

In [41]:
def leer_blob_en_memoria(bucket_name, blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    pdf_content = blob.download_as_bytes()
    return pdf_content

def extraer_texto_de_pdf_bytes(pdf_bytes):
    pdf_file = BytesIO(pdf_bytes)
    reader = PdfReader(pdf_file)
    text = ''
    for page in reader.pages:
        text += page.extract_text() + '\n'
    return text

def procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name):
    pdf_bytes = leer_blob_en_memoria(bucket_name, blob_name)
    content = extraer_texto_de_pdf_bytes(pdf_bytes)
    return content

def listar_pdfs(bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blobs = bucket.list_blobs()

    # Filtra los archivos PDF y guarda sus URLs completas en la lista
    listado_pdfs = [f"gs://{bucket_name}/{blob.name}" for blob in blobs if blob.name.endswith('.pdf') and not blob.name.startswith('resoluciones/')]
    return listado_pdfs

def dividir_lista_pdfs(listado_pdfs):
    total = len(listado_pdfs)
    tamaño_parte = total // 5
    
    parte1 = listado_pdfs[:tamaño_parte]
    parte2 = listado_pdfs[tamaño_parte:2*tamaño_parte]
    parte3 = listado_pdfs[2*tamaño_parte:3*tamaño_parte]
    parte4 = listado_pdfs[3*tamaño_parte:4*tamaño_parte]
    parte5 = listado_pdfs[4*tamaño_parte:]
    
    return parte1, parte2, parte3, parte4, parte5

In [35]:
bucket_name = 'tfm_javi'

In [7]:
# Definir los esquemas de respuesta
response_schemas = [
    ResponseSchema(name="numero_expediente", description="Número de Expediente, ejemplo SD2023/0000046"),
    ResponseSchema(name="resolucion", description="Una de las siguientes opciones: negada_con_oposicion, negada_sin_oposicion, aprobada_sin_oposicion, aprobada_con_oposición"),
    ResponseSchema(name="numero_de_resolución", description="Número entero de la resolución, ejemplo 2195"),
    ResponseSchema(name="denominacion", description="Nombre de la empresa que solicita el registro de la marca"),
    ResponseSchema(name="vigencia", description="Fecha en que expira la vigencia del registro, o texto si es negada o vencida"),
    ResponseSchema(name="titular", description="Titular de la marca que intenta registrar"),
    ResponseSchema(name="clase", description="Número o lista de números de la Clasificación Internacional de Niza, por ejemplo [42, 35, 27]"),
    ResponseSchema(name="gaceta", description="Número de la gaceta de Propiedad Industrial donde se publica"),
    ResponseSchema(name="tipo", description="Tipo de registro que se intenta realizar, por ejemplo Mixta, Nominativa, Figurativa"),
    ResponseSchema(name="fecha_solicitud", description="Fecha de presentación de la solicitud"),
    ResponseSchema(name="fecha_resolucion", description="Fecha de resolución"),
    ResponseSchema(name="nombre_opositor", description="Nombre de la empresa que se opone a la publicación"),
    ResponseSchema(name="signo_opositor_opositores", description="Signo o signos de los opositores en conflicto"),
    ResponseSchema(name="argumento_oposición", description="Argumentos en los que se basa para oponerse al registro y artículos en los que se apoya"),
    ResponseSchema(name="explicacion_argumentos_oposicion", description="Breve resumen y explicación de los argumentos de la oposición"),
    ResponseSchema(name="resolucion_organismo", description="Resolución del organismo competente, por ejemplo: 'DENIEGA el registro de la marca PAPELES LA FAVORITA (Mixta)'")
]


In [8]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
# Escapar las llaves para evitar la interpretación de variables
format_instructions = format_instructions.replace("{", "{{").replace("}", "}}")

In [9]:
system_prompt = """
Eres un Experto abogado Colombiano en analizar resoluciones del SIC (Superintendencia de Industria y Comercio de Colombia),en el ámbito de registro de marcas y lemas.
quiero que extraigas el numero de Expediente, la resolución del conflicto, el numero de la resolución, el nombre de la marca que intenta registrarse, el titular que intenta registrar la marca, el numero de clase que intenta registrar, 
el numero de la gaceta en que ha sido publicada, la fecha de solicitud de registro, nombre de la empresa opositora, el titular de la empresa que se opone si aparece, y los argumentos de derecho en los que se apoya el opositor.
"""
human_prompt = f"""Extrae la información indicada en DATOS a partir del TEXTO de la resolución

TEXT
---
\n\n{{contenido_pdf}}
---

DATOS
{format_instructions}
"""


In [10]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template(human_prompt),
])

In [11]:
# Define tu LLM 
llm = ChatVertexAI(
    model="gemini-1.5-pro-001"
      #verificar la relacion megas tokens  
)

In [12]:
chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)

C:\Users\javie\AppData\Local\Temp\ipykernel_9940\2259571032.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)


In [64]:
res = chain.invoke({"contenido_pdf": contenido_pdf})

# Parsear la respuesta
#parsed_output = output_parser.parse(res)

{'contenido_pdf': 'REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE INDUSTRIA Y COMERCIO \nResolución N° 8535\nRef. Expediente N° SD2021/0041386\nPágina 1 de 5Ref. Reg. Internacional N° 1588498\nPor la cual se niega un registro \nLA DIRECTORA DE SIGNOS DISTINTIVOS (E)\nen ejercicio de sus facultades legales y,\nCONSIDERANDO\nPRIMERO: Que por escrito presentado ante la Oficina Internacional el día 27 de \nnoviembre de 2020 y radicada ante la Superintendencia de Industria y Comercio el día 29 \nde abril de 2021, ORICA INTERNATIONAL PTE. LTD., solicitó el registro de la Marca 4D \n(Mixta) para distinguir productos y servicios comprendidos en las clases 1, 7, 9, 12, 13, 37 \ny 42 de la Clasificación Internacional de Niza1. \nSEGUNDO: Que publicado en la Gaceta de Propiedad Industrial No. 976, no se \npresentaron oposiciones por parte de terceros.\nTERCERO: Que atendiendo las disposiciones establecidas en la Decisión 486 de la \n1 1: Productos químicos utilizados en las industrias de minería, exp

In [65]:
res

{'contenido_pdf': 'REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE INDUSTRIA Y COMERCIO \nResolución N° 8535\nRef. Expediente N° SD2021/0041386\nPágina 1 de 5Ref. Reg. Internacional N° 1588498\nPor la cual se niega un registro \nLA DIRECTORA DE SIGNOS DISTINTIVOS (E)\nen ejercicio de sus facultades legales y,\nCONSIDERANDO\nPRIMERO: Que por escrito presentado ante la Oficina Internacional el día 27 de \nnoviembre de 2020 y radicada ante la Superintendencia de Industria y Comercio el día 29 \nde abril de 2021, ORICA INTERNATIONAL PTE. LTD., solicitó el registro de la Marca 4D \n(Mixta) para distinguir productos y servicios comprendidos en las clases 1, 7, 9, 12, 13, 37 \ny 42 de la Clasificación Internacional de Niza1. \nSEGUNDO: Que publicado en la Gaceta de Propiedad Industrial No. 976, no se \npresentaron oposiciones por parte de terceros.\nTERCERO: Que atendiendo las disposiciones establecidas en la Decisión 486 de la \n1 1: Productos químicos utilizados en las industrias de minería, exp

In [38]:
# Obtener la lista de PDFs
listado_pdfs = listar_pdfs(bucket_name)

In [42]:
len(listado_pdfs)

12786

In [43]:
parte1, parte2, parte3, parte4, parte5 = dividir_lista_pdfs(listado_pdfs)

In [53]:
parte1

['gs://tfm_javi/CCO-14258446.pdf',
 'gs://tfm_javi/CCO-15006549.pdf',
 'gs://tfm_javi/CCO-15030961.pdf',
 'gs://tfm_javi/CCO-15036335.pdf',
 'gs://tfm_javi/CCO-15036379.pdf',
 'gs://tfm_javi/CCO-15068367.pdf',
 'gs://tfm_javi/CCO-15074926.pdf',
 'gs://tfm_javi/CCO-15078424.pdf',
 'gs://tfm_javi/CCO-15115149.pdf',
 'gs://tfm_javi/CCO-15121325.pdf',
 'gs://tfm_javi/CCO-15121347.pdf',
 'gs://tfm_javi/CCO-15150860.pdf',
 'gs://tfm_javi/CCO-15169411.pdf',
 'gs://tfm_javi/CCO-15169425.pdf',
 'gs://tfm_javi/CCO-15175257.pdf',
 'gs://tfm_javi/CCO-15175281.pdf',
 'gs://tfm_javi/CCO-15213225.pdf',
 'gs://tfm_javi/CCO-15213227.pdf',
 'gs://tfm_javi/CCO-15213269.pdf',
 'gs://tfm_javi/CCO-15233002.pdf',
 'gs://tfm_javi/CCO-15233023.pdf',
 'gs://tfm_javi/CCO-15233036.pdf',
 'gs://tfm_javi/CCO-15244609.pdf',
 'gs://tfm_javi/CCO-15244658.pdf',
 'gs://tfm_javi/CCO-15244663.pdf',
 'gs://tfm_javi/CCO-15244697.pdf',
 'gs://tfm_javi/CCO-15244792.pdf',
 'gs://tfm_javi/CCO-15251313.pdf',
 'gs://tfm_javi/CCO-

In [51]:
# Inicializar una lista para almacenar los resultados
resultados = []


In [59]:
 for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

NotFound: 404 GET https://storage.googleapis.com/download/storage/v1/b/tfm_javi/o/g?alt=media: No such object: tfm_javi/g: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)

## Errores de analisis
realizamos en la terminal cat error_log.txt que nos da el resultado de:

error al procesar resoluciones/NCO-SD2022-0073142.pdf

error al procesar resoluciones/NCO-SD2022-0113402.pdf

error al procesar resoluciones/SD2022-0017210.pdf

vamos  a realizar un intento individual con esos documentos

In [57]:
listado_pdfs_error = f"gs://{bucket_name}/CSO-SD2017-0001227.pdf"
    

In [58]:
listado_pdfs_error

'gs://tfm_javi/CSO-SD2017-0001227.pdf'

In [96]:
listado_pdfs_error

('gs://tfm_javi/resoluciones/NCO-SD2022-0073142.pdf',
 'gs://tfm_javi/resoluciones/NCO-SD2022-0113402.pdf',
 'gs://tfm_javi/resoluciones/SD2022-0017210.pdf')

In [97]:
 for pdf_uri in listado_pdfs_error:
     blob_name = pdf_uri.replace(f"gs://{bucket_name}/", "")
     contenido_pdf = procesar_pdf_desde_gcs_en_memoria(bucket_name, blob_name)
    
      # Verificar si el contenido del PDF no está vacío
     if contenido_pdf.strip():
         try:
              # Invocar la cadena con el contenido del PDF
             res = chain.invoke({"contenido_pdf": contenido_pdf})
              #Agregar el resultado a la lista de resultados
             resultados.append(res)
              #Imprimir el resultado
             print(f"Resultado para {blob_name}:\n{res}\n")
         except Exception as e:
            error_message = f"Ocurrió un error al procesar {blob_name}: {e}"
            print(error_message)
            # Registrar el error en el log
            logging.error(error_message)
     else:
        error_message = f"El contenido de {blob_name} está vacío o no se pudo extraer texto."
        print(error_message)
        # Registrar el error en el log
        logging.error(error_message)

Resultado para resoluciones/NCO-SD2022-0073142.pdf:
{'contenido_pdf': 'REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE INDUSTRIA Y COMERCIO \nResolución N° 16207\nRef. Expediente N° SD2022/0073142\nRef. Reg. Internacional N° 1673357\nPágina 1 de 48Por la cual se decide una solicitud de registro \nLA DIRECTORA DE SIGNOS DISTINTIVOS (E)\nen ejercicio de sus facultades legales y,\nCONSIDERANDO\nQue por escrito presentado ante la Oficina Internacional el día 30 de marzo de 2022 y \nradicada ante la Superintendencia de Industria y Comercio el día 21 de julio de 2022, YS \nFAMILY MAYORISTA 2021 SLU, solicitó el registro de la marca FAMILY (Mixta) para \ndistinguir productos comprendidos en las clases 8, 20, 21 y 24 de la Clasificación \nInternacional de Niza1. \nQue publicado en la Gaceta de Propiedad Industrial No. 974, PRODUCTOS FAMILIA S.A., \npresentó oposición en contra de las clases 20, 21 y 24 con fundamento en las causales \nde irregistrabilidad establecidas en los literales a) y h) del a

Parece que esta vez si que ha procesado bien los tres archivos que diéron error, vamos a construir un df con los resultados

In [60]:
df_resultados = pd.DataFrame(resultados)

In [61]:
df_resultados

,contenido_pdf,text
0,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IN...,"{'numero_expediente': '14258446', 'resolucion'..."
1,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': '15006549', 'resolucion'..."
2,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': '15030961', 'resolucion'..."
3,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': '15036335', 'resolucion'..."
4,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': '15036379', 'resolucion'..."
...,...,...
2551,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': 'SD2019/0040431', 'resol..."
2552,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': 'SD2021/0031995', 'resol..."
2553,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': 'SD2021/0085336', 'resol..."
2554,REPÚBLICA DE COLOMBIA\nSUPERINTENDENCIA DE IND...,"{'numero_expediente': 'SD2017/0025025', 'resol..."


In [62]:
pd.set_option('display.max_columns', None)

In [63]:
df_ai_annotations_full_1 = pd.DataFrame.from_records(df_resultados['text'])

In [64]:
df_ai_annotations_full_1

,numero_expediente,resolucion,numero_de_resolución,denominacion,vigencia,titular,clase,gaceta,tipo,fecha_solicitud,fecha_resolucion,nombre_opositor,signo_opositor_opositores,argumento_oposición,explicacion_argumentos_oposicion,resolucion_organismo
0,14258446,aprobada_con_oposición,82711,ChaCheer,20 de marzo de 2023.,"QIAQIA FOOD CO., LTD.","[29, 30]",719,Mixta,24 de noviembre de 2014,29 de noviembre de 2016,INVERSIONES CHEERS DEL CARIBE S.A.,CHEERS SINCE 2009,Basada en el literal a) del artículo 136 de la...,La oposición argumenta que existe un alto grad...,Declara infundada la oposición interpuesta por...
1,15006549,aprobada_con_oposición,50676,Granini,26 de agosto de 2030,ECKES-GRANINI GROUP GMBH,32,726,Mixta,14 de enero de 2015,26 de agosto de 2020,TORRECAFE AGUILA ROJA [&] CIA S.A.,GRANINO\n(Nominativa),La marca solicitada GRANINI es confundible con...,El opositor argumenta que existe riesgo de con...,Se declara infundada la oposición interpuesta ...
2,15030961,aprobada_con_oposición,3429,SECTRA,16 de mayo de 2024,SECTRA AB,"[9, 10, 16, 35, 38, 41, 42, 44]",755,Nominativa,12 de febrero de 2015,3 de febrero de 2017,"ESPECIALISTAS EN RADIOLOGIA S.A.S, SIXTRA CHIL...","ESPECTRA, SIXTRA",Los opositores argumentan que la marca solicit...,Los opositores consideran que existe un alto r...,La Superintendencia de Industria y Comercio de...
3,15036335,aprobada_con_oposición,19923,BODEGAS RODA SELA,26 de junio de 2022,"BODEGAS RODA, S.A.",33,767,Mixta,5 de diciembre de 2014,24 de abril de 2017,BODEGAS DE MOSELA LTDA.,BODEGAS DE MOSELA (Nominativa),El opositor argumenta que el signo solicitado ...,La oposición se fundamenta en la similitud ent...,La Superintendencia de Industria y Comercio re...
4,15036379,aprobada_con_oposición,10639,bag,29 de octubre de 2024,J. Luscombe Associates Limited,"[9, 14, 18, 25, 35]",736,Mixta,19 de febrero de 2015,8 de marzo de 2017,OUR BAG S.A.S.,OUR BAG y b. bag,Artículo 136 literal a) de la Decisión 486 de ...,El opositor argumenta que la marca solicitada ...,Se declara infundada la oposición y se concede...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2551,SD2019/0040431,aprobada_con_oposición,220779,MI WALLET,20 de noviembre de 2028,XIAOMI INC.,"[9, 36]",863,Mixta,20 de noviembre de 2018,17 de octubre de 2019,UALET S.A. COMISIONISTA DE BOLSA,UALET,La oposición se basa en las causales de irregi...,El opositor argumenta que MI WALLET es similar...,Se declara INFUNDADA la oposición interpuesta ...
2552,SD2021/0031995,aprobada_con_oposición,2853,ALPECIN HYBRID,15 de mayo de 2028,DR. KURT WOLFF GMBH & CO. KG,"[3, 5]",929,Mixta,17 de marzo de 2021,31 de enero de 2022,GLAUKOS CORPORATION,IBRID (Nominativo),La marca solicitada ALPECIN HYBRID es similar ...,El opositor argumenta que la similitud fonétic...,Se declara infundada la oposición interpuesta ...
2553,SD2021/0085336,aprobada_con_oposición,82395,Fini,04/03/2031,"Sánchez Cano, S.A.","[5, 29, 30, 35, 36, 41]",963,Mixta,04/03/2021,23/11/2022,Finppi Colombia SA.S.\nAlimentos Finca S.A.S.,"Finppi, FINCA MININO, FINCA MIRRINGO, F FINCA,...",Los opositores argumentan que la marca solicit...,Los argumentos de la oposición se basan en la ...,Se deniega la extensión de la notoriedad de la...
2554,SD2017/0025025,aprobada_sin_oposicion,64235,MR-174 Watching the World \n. Designed by Mits...,12 de agosto de 2019,"MITSUI CHEMICALS, INC.",9,None,Mixta,None,9 de octubre de 2017,None,None,None,None,Conceder el registro de la Marca MR-174 Watchi...


In [65]:
df_ai_annotations_full_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   numero_expediente                 2556 non-null   object
 1   resolucion                        2556 non-null   object
 2   numero_de_resolución              2556 non-null   object
 3   denominacion                      2404 non-null   object
 4   vigencia                          2535 non-null   object
 5   titular                           2556 non-null   object
 6   clase                             2555 non-null   object
 7   gaceta                            588 non-null    object
 8   tipo                              2556 non-null   object
 9   fecha_solicitud                   1399 non-null   object
 10  fecha_resolucion                  2550 non-null   object
 11  nombre_opositor                   584 non-null    object
 12  signo_opositor_oposi

In [104]:
df_ai_annotations["argumento_oposición"]

0                                                    N/A
1                                                   None
2                                                   None
3                                                   None
4                                                   None
                             ...                        
427    La marca solicitada FleXeDrive (Nominativa) pa...
428                                                 None
429    El opositor argumenta que el signo solicitado ...
430    Artículo 136 literal a) de la Decisión 486 de ...
431    Artículo 136 literal a) de la Decisión 486 de ...
Name: argumento_oposición, Length: 432, dtype: object

In [67]:
df_ai_annotations_full_1.to_csv("datasetia_full_1.csv")

In [6]:
df = pd.read_csv("datasetia.csv")

In [7]:
df = df.drop(columns=['Unnamed: 0'])

In [8]:
df

,numero_expediente,resolucion,numero_de_resolución,denominacion,vigencia,titular,clase,gaceta,tipo,fecha_solicitud,fecha_resolucion,nombre_opositor,signo_opositor_opositores,argumento_oposición,explicacion_argumentos_oposicion,resolucion_organismo
0,SD2022/0076224,aprobada_sin_oposicion,3730,NESCAFÉ Dolce Gusto neo,15 de julio de 2031,Société des Produits Nestlé S.A.,11,981.0,Mixta,28 de julio de 2022,6 de febrero de 2023,NaN,NaN,NaN,NaN,Conceder el registro de la Marca NESCAFÉ Dolce...
1,SD2022/0076226,aprobada_sin_oposicion,3738,DOLCE GUSTO NEO,4 de enero de 2032.,Société des Produits Nestlé S.A.,"[29, 30]",981.0,Nominativa,28 de julio de 2022,6 de febrero de 2023,NaN,NaN,NaN,NaN,Conceder el registro de la Marca DOLCE GUSTO N...
2,SD2022/0076227,aprobada_sin_oposicion,3740,NESCAFÉ Dolce Gusto neo,4 de enero de 2032,Société des Produits Nestlé S.A.,"[29, 30]",981.0,Mixta,28 de julio de 2022,6 de febrero de 2023,NaN,NaN,NaN,NaN,Conceder el registro de la Marca NESCAFÉ Dolce...
3,SD2022/0078960,aprobada_sin_oposicion,3092,MotionSync,02.04.2032,"Honor Device Co., Ltd.",9,976.0,Nominativa,4 de agosto de 2022,1 de febrero de 2023,NaN,NaN,NaN,NaN,Conceder el registro de la Marca MotionSync (N...
4,SD2022/0081645,aprobada_sin_oposicion,3094,Tranter FullServ,04.05.2032,Tranter International AB,"[37, 42]",976.0,Nominativa,11/08/2022,01/02/2023,NaN,NaN,NaN,NaN,Conceder el registro de la Marca Tranter FullS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,SD2022/0017139,negada_sin_oposicion,6466,FleXeDrive,NaN,SEW-EURODRIVE GmbH & Co KG,7,981.0,Nominativa,14 de diciembre de 2021,20 de febrero de 2023,"HELMERICH & PAYNE, INC.",FLEXDRIVE,La marca solicitada FleXeDrive (Nominativa) pa...,El argumento principal se basa en la similitud...,Niega el registro de la marca FleXeDrive (Nomi...
428,SD2022/0017171,aprobada_sin_oposicion,3282,AgroVision,15.10.2031,Agrovision B.V.,"[9, 35, 41, 42]",982.0,Mixta,24 de febrero de 2022,2 de febrero de 2023,NaN,NaN,NaN,NaN,Conceder el registro de la Marca AgroVision (M...
429,SD2022/0073142,negada_con_oposicion,16207,FAMILY,NaN,YS FAMILY MAYORISTA 2021 SLU,"[8, 20, 21, 24]",974.0,Mixta,30 de marzo de 2022,30 de marzo de 2023,PRODUCTOS FAMILIA S.A.,"FAMILIA, familia, familia miggo, mi familia, G...",El opositor argumenta que el signo solicitado ...,La oposición se fundamenta en que el signo sol...,Se deniega la solicitud de registro de la marc...
430,SD2022/0113402,negada_con_oposicion,64163,AGROGAIN,No aplica,SEA6 ENERGY PRIVATE LIMITED,1,990.0,Mixta,29 de julio de 2022,20 de octubre de 2023,AGRO GRAIN S.A.,AGRO GRAIN S.A. (mixta),Artículo 136 literal a) de la Decisión 486 de ...,La oposición se basa en la similitud entre las...,Se declara fundada la oposición interpuesta po...


***JDS:*** *veo haciendo una exploración que XING FA es una marca que al parecer ya esta registrada, y que la SIC ha entrado de oficio en un caso (expediente SD2022/0011834) porque entra en conflicto con esa marca, hago una exploración muy basica a ver si es uno de los casos que ya hemos procesado y aparentemete no, es posible que si que este pero habria que normalizarlo a minusculas etc, pero tambien es muy posible que no este al ser solo 419 elementos del total, habria que hacer un merge con la bbdd del sql que nos dieron de sysmarck*

In [21]:
df[df["denominacion"].str.contains("XING FA", case=False, na=False)]

,numero_expediente,resolucion,numero_de_resolución,denominacion,vigencia,titular,clase,gaceta,tipo,fecha_solicitud,fecha_resolucion,nombre_opositor,signo_opositor_opositores,argumento_oposición,explicacion_argumentos_oposicion,resolucion_organismo
